# Tag 8: Model Evaluation & Ethisches AI 🎯

**Dauer:** 90 Minuten  
**Datensatz:** Titanic Passenger Data

## Lernziele
- ✅ Bias vs. Variance Tradeoff verstehen
- ✅ Overfitting und Underfitting erkennen
- ✅ Klassifikationsmetriken anwenden (Accuracy, Precision, Recall, F1)
- ✅ Confusion Matrix interpretieren
- ✅ ROC/AUC Kurven verstehen
- ✅ Type I vs. Type II Errors unterscheiden
- ✅ Ethisches AI & Trustworthy AI Prinzipien kennen

---

## 1️⃣ Setup und Datenvorbereitung (5 min)

### Warum Model Evaluation?

Ein Machine Learning Modell zu bauen ist nur der erste Schritt. Die wichtigste Frage ist: **Wie gut ist unser Modell wirklich?**

**Heute lernen wir:**
- 📊 Modell-Performance objektiv messen
- 🎯 Erkennen wann Modelle zu simpel oder zu komplex sind
- 📈 Die richtigen Metriken für verschiedene Probleme wählen
- 🤖 Ethische AI-Systeme entwickeln

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns

# Machine Learning libraries
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_curve, roc_auc_score,
    mean_squared_error, mean_absolute_error, r2_score
)

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully!")

In [ ]:
# Load Titanic dataset
df = sns.load_dataset('titanic')
print(f"Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"Survival rate: {df['survived'].mean()*100:.1f}%")
df.head()

In [ ]:
# Daten vorbereiten
df_clean = df.copy()
df_clean['age'].fillna(df_clean['age'].median(), inplace=True)
df_clean['embarked'].fillna(df_clean['embarked'].mode()[0], inplace=True)
df_clean['fare'].fillna(df_clean['fare'].median(), inplace=True)

# Features erstellen
df_clean['sex_encoded'] = df_clean['sex'].map({'male': 1, 'female': 0})
df_clean['family_size'] = df_clean['sibsp'] + df_clean['parch'] + 1
df_clean['is_alone'] = (df_clean['family_size'] == 1).astype(int)

# Finale Features
features = ['pclass', 'sex_encoded', 'age', 'fare', 'family_size', 'is_alone']
X = df_clean[features]
y = df_clean['survived']

# Train-Test Split (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("✓ Daten vorbereitet!")
print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

---
## 2️⃣ Bias-Variance Tradeoff (7 min)

### Die Goldene Regel des Machine Learning

**NIEMALS auf Trainingsdaten testen!**

**Analogie - Prüfungsvorbereitung:**
- **Training Set:** Übungsaufgaben zum Lernen
- **Test Set:** Die echte Prüfung mit neuen Fragen

**Bias (Verzerrung):** Wie weit liegen Vorhersagen im Durchschnitt daneben?
- ⬆️ Hoher Bias = **Underfitting** (Modell zu simpel)
- Modell übersieht wichtige Muster

**Variance (Varianz):** Wie stark variieren Vorhersagen bei unterschiedlichen Trainingsdaten?
- ⬆️ Hohe Variance = **Overfitting** (Modell zu komplex)
- Modell merkt sich Trainingsdaten inkl. Rauschen

```
High Bias          Sweet Spot          High Variance
(Underfitting)                         (Overfitting)
Zu simpel          ✅ Balance           Zu komplex
```

### 🎯 In-Lecture Übung 2.1: Bias vs. Variance Identifizieren (5 min)

**Szenario:** Drei ML-Modelle wurden trainiert:

**Modell A:**
- Training Accuracy: 65%
- Test Accuracy: 64%

**Modell B:**
- Training Accuracy: 99%
- Test Accuracy: 72%

**Modell C:**
- Training Accuracy: 82%
- Test Accuracy: 79%

**Aufgaben:**
1. Welches Modell hat **Underfitting** (High Bias)?
2. Welches Modell hat **Overfitting** (High Variance)?
3. Welches Modell ist am besten balanciert?

```python
# TODO: Schreibe deine Antworten hier
# 1. Underfitting: Modell ___
# 2. Overfitting: Modell ___
# 3. Beste Balance: Modell ___
```

In [ ]:
# TODO: Teste deine Antworten
# Antworten:
# 1. Modell A (beide niedrig = zu simpel)
# 2. Modell B (riesiger Gap = merkt sich Trainingsdaten)
# 3. Modell C (kleiner Gap, gute Performance)

---
## 3️⃣ Klassifikationsmetriken (10 min)

### Confusion Matrix - Die Grundlage aller Metriken

Eine **Confusion Matrix** zeigt, wie oft ein Klassifikationsmodell richtig vs. falsch liegt:

```
                 Predicted
                 0    1
Actual  0       TN   FP
        1       FN   TP
```

**Vier Quadranten:**
- **True Positive (TP):** Richtig als positiv erkannt (z.B. Überlebende korrekt vorhergesagt)
- **True Negative (TN):** Richtig als negativ erkannt (z.B. Nicht-Überlebende korrekt vorhergesagt)
- **False Positive (FP):** Fälschlicherweise als positiv erkannt (**Type I Error**)
- **False Negative (FN):** Fälschlicherweise als negativ erkannt (**Type II Error**)

### Die wichtigsten Metriken:

**1. Accuracy (Genauigkeit):**
```
Accuracy = (TP + TN) / (TP + TN + FP + FN)
```
Wie viele Vorhersagen waren insgesamt richtig?

**2. Precision (Präzision):**
```
Precision = TP / (TP + FP)
```
Von allen als "positiv" vorhergesagten, wie viele waren wirklich positiv?

**3. Recall (Sensitivität):**
```
Recall = TP / (TP + FN)
```
Von allen echten Positiven, wie viele haben wir gefunden?

**4. F1-Score:**
```
F1 = 2 × (Precision × Recall) / (Precision + Recall)
```
Harmonisches Mittel von Precision und Recall.

### 🎯 In-Lecture Übung 3.1: Confusion Matrix interpretieren (7 min)

**Gegeben:** Ein Modell zur Titanic-Überlebensvorhersage

```
Confusion Matrix:
                Predicted
                Dead  Survived
Actual Dead      90      15
       Survived  10      64
```

**Aufgaben:**
1. Berechne **Accuracy**
2. Berechne **Precision** (für "Survived")
3. Berechne **Recall** (für "Survived")
4. Was bedeutet diese Confusion Matrix für unser Modell?

In [ ]:
# TODO: Berechne die Metriken
TP = 64  # Survived richtig vorhergesagt
TN = 90  # Dead richtig vorhergesagt
FP = 15  # Dead als Survived vorhergesagt
FN = 10  # Survived als Dead vorhergesagt

# Aufgabe 1: Accuracy
accuracy = # TODO: Deine Formel hier

# Aufgabe 2: Precision
precision = # TODO: Deine Formel hier

# Aufgabe 3: Recall
recall = # TODO: Deine Formel hier

print(f"Accuracy: {accuracy:.2%}")
print(f"Precision: {precision:.2%}")
print(f"Recall: {recall:.2%}")

---
## 4️⃣ Type I vs. Type II Errors (5 min)

### Medizinisches Beispiel - COVID-19 Test:

**Type I Error (False Positive, α):**
- Fehler: Gesunde Person wird als "positiv" diagnostiziert
- Konsequenz: Unnötige Quarantäne, Angst
- **"False Alarm"**

**Type II Error (False Negative, β):**
- Fehler: Kranke Person wird als "negativ" diagnostiziert
- Konsequenz: Keine Behandlung, Verbreitung der Krankheit
- **"Missed Detection"**

**Welcher Fehler ist schlimmer?** → Hängt vom Kontext ab!

- **Spam Filter:** FP schlimmer (wichtige E-Mail geht verloren)
- **Krebs-Diagnose:** FN schlimmer (Krankheit wird übersehen)
- **Titanic:** FN schlimmer (Überlebenschance wird falsch eingeschätzt)

### 🎯 In-Lecture Übung 4.1: Fehlertypen Identifizieren (3 min)

**Für folgende Szenarien:** Welcher Fehler ist gefährlicher - Type I oder Type II?

1. **Betrugserkennung bei Kreditkarten**
   - Type I: Legitime Transaktion wird blockiert
   - Type II: Betrügerische Transaktion wird durchgelassen
   - Schlimmer? ___________

2. **Autopilot Fußgängererkennung**
   - Type I: Objekt wird fälschlicherweise als Fußgänger erkannt
   - Type II: Echter Fußgänger wird nicht erkannt
   - Schlimmer? ___________

3. **Job-Bewerbung Vorauswahl (AI)**
   - Type I: Ungeeigneter Kandidat wird eingeladen
   - Type II: Geeigneter Kandidat wird abgelehnt
   - Schlimmer? ___________

In [ ]:
# TODO: Diskutiere mit deinem Nachbarn
# Antworten:
# 1. Type II (Betrug wird nicht erkannt → Geldverlust)
# 2. Type II (Fußgänger wird übersehen → Unfall!)
# 3. Kontext-abhängig (oft Type II → Talente gehen verloren)

---
## 5️⃣ ROC Curve & AUC (10 min)

### ROC Curve - Receiver Operating Characteristic

Die **ROC Curve** visualisiert die Performance eines Klassifikators über alle möglichen Schwellenwerte.

**Achsen:**
- **X-Achse:** False Positive Rate (FPR) = FP / (FP + TN)
- **Y-Achse:** True Positive Rate (TPR) = TP / (TP + FN) = Recall

**AUC - Area Under the Curve:**
- **AUC = 1.0:** Perfekter Klassifikator
- **AUC = 0.5:** Zufälliges Raten (Diagonale)
- **AUC < 0.5:** Schlechter als Zufall

**Interpretation:**
- Je näher die Kurve an der linken oberen Ecke, desto besser
- AUC gibt eine einzelne Zahl für die Gesamtperformance

### 🎯 In-Lecture Übung 5.1: ROC Curve erstellen (7 min)

**Aufgabe:** Erstelle eine ROC Curve für ein Logistic Regression Modell auf dem Titanic-Datensatz.

**Schritte:**
1. Trainiere ein Logistic Regression Modell
2. Berechne Vorhersage-Wahrscheinlichkeiten
3. Erstelle ROC Curve
4. Berechne AUC Score

In [ ]:
# TODO: Erstelle ROC Curve
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, RocCurveDisplay

# Modell trainieren
model = LogisticRegression(max_iter=1000)
# TODO: Trainiere das Modell

# Vorhersage-Wahrscheinlichkeiten
# TODO: Berechne y_pred_proba für die positive Klasse

# ROC Curve berechnen
# TODO: Verwende roc_curve(y_test, y_pred_proba)

# AUC berechnen
# TODO: Berechne auc(fpr, tpr)

# Visualisierung
# TODO: Plotte die ROC Curve mit matplotlib oder RocCurveDisplay

print(f"AUC Score: {roc_auc:.3f}")

---
## 6️⃣ Ethisches AI & Trustworthy AI (10 min)

### Warum Ethik in AI wichtig ist

**Real-World Probleme:**
- 🏥 **Healthcare:** Algorithmen mit Bias gegen bestimmte Ethnien
- 👮 **Polizei:** Gesichtserkennung mit höheren Fehlerraten bei dunkler Hautfarbe
- 💼 **Recruiting:** AI-Systeme bevorzugen männliche Kandidaten
- 💰 **Kredite:** Diskriminierung basierend auf Postleitzahl

### EU AI Act - Trustworthy AI Prinzipien:

1. **Human Agency & Oversight** - Menschliche Kontrolle
2. **Technical Robustness & Safety** - Zuverlässigkeit
3. **Privacy & Data Governance** - Datenschutz
4. **Transparency** - Nachvollziehbarkeit
5. **Diversity & Fairness** - Keine Diskriminierung
6. **Societal & Environmental Wellbeing** - Gesellschaftlicher Nutzen
7. **Accountability** - Verantwortlichkeit

### 🎯 In-Lecture Übung 6.1: Ethik-Szenario Analyse (8 min)

**Szenario:** Eine Universität entwickelt ein AI-System zur Bewertung von Studierenden-Bewerbungen.

**Gegebene Informationen:**
- Training Data: Letzte 10 Jahre akzeptierte Studierende
- Features: Noten, Alter, Geschlecht, Postleitzahl, Hobbies
- Accuracy: 89% auf Test-Daten

**Diskussionsfragen:**

1. **Welche Bias-Risiken siehst du in diesem System?**
   
   ___________________________________

2. **Welche EU AI Act Prinzipien könnten verletzt werden?**
   
   ___________________________________

3. **Wie könntest du das System fairer gestalten?**
   
   ___________________________________

4. **Sollte Geschlecht als Feature verwendet werden? Warum/nicht?**
   
   ___________________________________

In [ ]:
# TODO: Diskutiere in Gruppen (4-5 Minuten)
# Mögliche Probleme:
# - Historical Bias: System lernt alte Diskriminierungsmuster
# - Gender Bias: Geschlecht sollte irrelevant sein
# - Geographic Bias: Postleitzahl als Proxy für Einkommen
# - Fehlende Diversity & Fairness
# - Mangelnde Transparency (Black Box)

---
---
# 📝 Wiederholungsblock - Für Zuhause

**Ziel:** Vertiefe dein Verständnis von Model Evaluation durch praktische Übungen.

**Zeitaufwand:** 60-90 Minuten  
**Empfehlung:** Arbeite alle Blöcke durch, bevor du mit dem Abschlussprojekt startest!

---

## 📝 W1: Regression Metriken Deep Dive

### Aufgabe: Titanic Fare Vorhersage mit Regression Metrics

**Ziel:** Verstehe MAE, MSE, RMSE, R² durch praktische Anwendung.

**Regression Metriken Überblick:**

1. **MAE (Mean Absolute Error):**
   - Durchschnittlicher absoluter Fehler
   - Einheit: Gleiche wie Zielvariable
   - Robust gegen Ausreißer

2. **MSE (Mean Squared Error):**
   - Durchschnittlicher quadratischer Fehler
   - Bestraft große Fehler stärker
   - Empfindlich gegen Ausreißer

3. **RMSE (Root Mean Squared Error):**
   - Wurzel aus MSE
   - Gleiche Einheit wie Zielvariable
   - Interpretierbar

4. **R² (Coefficient of Determination):**
   - Anteil erklärter Varianz (0 bis 1)
   - 1.0 = perfekte Vorhersage
   - 0.0 = Mittelwert-Baseline

In [ ]:
# TODO: Regression Metriken praktisch anwenden
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Regression Problem: Fare Vorhersage
X_fare = df_clean[['pclass', 'age', 'family_size']].dropna()
y_fare = df_clean.loc[X_fare.index, 'fare']

# Train-Test Split
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_fare, y_fare, test_size=0.2, random_state=42)

# Modell trainieren
lr_model = LinearRegression()
# TODO: Trainiere das Modell

# Vorhersagen
# TODO: Berechne y_pred

# Metriken berechnen
mae = # TODO: mean_absolute_error(y_test_r, y_pred)
mse = # TODO: mean_squared_error(y_test_r, y_pred)
rmse = # TODO: np.sqrt(mse)
r2 = # TODO: r2_score(y_test_r, y_pred)

print(f"MAE: ${mae:.2f}")
print(f"MSE: ${mse:.2f}")
print(f"RMSE: ${rmse:.2f}")
print(f"R²: {r2:.3f}")

# Interpretation:
# - MAE: Durchschnittlich ${mae:.2f} Abweichung
# - RMSE: Bestraft große Fehler stärker
# - R²: {r2:.1%} der Varianz wird erklärt

## 📝 W2: Learning Curves & Model Complexity

### Aufgabe: Visualisiere Overfitting mit Learning Curves

**Learning Curves** zeigen wie sich Training- und Test-Performance mit mehr Daten entwickeln.

**Interpretation:**
- **Hoher Bias (Underfitting):** Beide Kurven konvergieren auf niedriger Performance
- **Hohe Variance (Overfitting):** Großer Gap zwischen Training und Test
- **Optimal:** Beide Kurven konvergieren auf hoher Performance

In [ ]:
# TODO: Erstelle Learning Curves
from sklearn.model_selection import learning_curve
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt

# Teste verschiedene max_depth Werte
depths = [2, 5, 10, 20]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

for idx, depth in enumerate(depths):
    model = DecisionTreeClassifier(max_depth=depth, random_state=42)
    
    # Learning Curve berechnen
    train_sizes, train_scores, test_scores = learning_curve(
        # TODO: Fülle die Parameter aus
        # model, X, y, cv=5, train_sizes=np.linspace(0.1, 1.0, 10)
    )
    
    # Durchschnitte berechnen
    # TODO: train_mean = np.mean(train_scores, axis=1)
    # TODO: test_mean = np.mean(test_scores, axis=1)
    
    # Plotten
    axes[idx].plot(train_sizes, train_mean, label='Training Score', marker='o')
    axes[idx].plot(train_sizes, test_mean, label='Test Score', marker='s')
    axes[idx].set_title(f'max_depth={depth}')
    axes[idx].set_xlabel('Training Set Size')
    axes[idx].set_ylabel('Accuracy')
    axes[idx].legend()
    axes[idx].grid(True)

plt.tight_layout()
plt.show()

# Frage: Welches max_depth ist optimal?

## 📝 W3: Multi-Model Comparison mit allen Metriken

### Aufgabe: Vergleiche 5 verschiedene Modelle umfassend

**Ziel:** Lerne wie man Modelle systematisch vergleicht und das Beste auswählt.

In [ ]:
# TODO: Vergleiche mehrere Modelle systematisch
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pandas as pd

# Modelle definieren
models = {
    'K-NN (k=5)': KNeighborsClassifier(n_neighbors=5),
    'Decision Tree': DecisionTreeClassifier(max_depth=5, random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(probability=True, random_state=42)
}

# Ergebnisse sammeln
results = []

for name, model in models.items():
    # TODO: Trainiere jedes Modell
    # TODO: Mache Vorhersagen
    # TODO: Berechne alle Metriken
    
    # Metriken berechnen
    acc = # TODO
    prec = # TODO
    rec = # TODO
    f1 = # TODO
    auc = # TODO
    
    results.append({
        'Model': name,
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1-Score': f1,
        'AUC': auc
    })

# Ergebnisse als DataFrame
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('F1-Score', ascending=False)

print(results_df.to_string(index=False))

# Frage: Welches Modell ist am besten? Warum?

## 📝 W4: Hyperparameter Tuning mit GridSearchCV

### Aufgabe: Finde optimale Hyperparameter für Random Forest

**Grid Search:** Systematisches Testen aller Hyperparameter-Kombinationen

**Cross-Validation:** Mehrfache Train-Test Splits für robuste Bewertung

In [ ]:
# TODO: Hyperparameter Tuning mit GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Hyperparameter Grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# GridSearchCV
rf_model = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(
    # TODO: Fülle die Parameter aus
    # estimator=rf_model,
    # param_grid=param_grid,
    # cv=5,
    # scoring='f1',
    # n_jobs=-1
)

# Training
# TODO: grid_search.fit(X_train, y_train)

# Beste Parameter
print("Beste Parameter:", grid_search.best_params_)
print(f"Beste F1-Score (CV): {grid_search.best_score_:.3f}")

# Test auf Test-Set
best_model = grid_search.best_estimator_
# TODO: y_pred = best_model.predict(X_test)
# TODO: Berechne Test-Metriken

print(f"\nTest Set Performance:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.3f}")

## 📝 W5: Fairness & Bias Detection im Titanic-Modell

### Aufgabe: Analysiere ob dein Modell fair gegenüber verschiedenen Gruppen ist

**Fairness Metrics:**
- **Demographic Parity:** Vorhersagerate sollte über Gruppen gleich sein
- **Equal Opportunity:** TPR sollte über Gruppen gleich sein
- **Equalized Odds:** TPR und FPR sollten über Gruppen gleich sein

In [ ]:
# TODO: Fairness-Analyse nach Geschlecht
from sklearn.metrics import confusion_matrix

# Trainiere ein Modell (z.B. Logistic Regression)
lr_model = LogisticRegression(max_iter=1000, random_state=42)
# TODO: Trainiere das Modell
# TODO: Mache Vorhersagen auf Test-Set

# Analysiere nach Geschlecht
genders = ['male', 'female']

for gender in genders:
    # Filter Test-Daten nach Geschlecht
    gender_mask = df_clean.loc[X_test.index, 'sex'] == gender
    y_test_gender = y_test[gender_mask]
    y_pred_gender = y_pred[gender_mask]
    
    # Confusion Matrix
    cm = confusion_matrix(y_test_gender, y_pred_gender)
    tn, fp, fn, tp = cm.ravel()
    
    # Metriken berechnen
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0  # Recall
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    
    print(f"\n{gender.upper()} Group:")
    print(f"  Sample Size: {len(y_test_gender)}")
    print(f"  True Positive Rate (Recall): {tpr:.3f}")
    print(f"  False Positive Rate: {fpr:.3f}")
    print(f"  Precision: {precision:.3f}")
    print(f"  Predicted Survival Rate: {y_pred_gender.mean():.3f}")

# Diskussion: Ist das Modell fair? Welche Unterschiede siehst du?

---
## 🎯 Zusammenfassung & Reflexion

### Haupterkenntnisse von Tag 8:

✅ **Bias-Variance Tradeoff:** Balance zwischen Underfitting und Overfitting  
✅ **Klassifikationsmetriken:** Accuracy, Precision, Recall, F1-Score, AUC  
✅ **Regression Metriken:** MAE, MSE, RMSE, R²  
✅ **Confusion Matrix:** Basis für alle Klassifikationsmetriken  
✅ **Type I vs II Errors:** Kontext bestimmt welcher Fehler schlimmer ist  
✅ **ROC/AUC:** Visualisierung der Modellperformance  
✅ **Ethisches AI:** Fairness, Transparency, Accountability

### Wichtig fürs Abschlussprojekt:

- **Wähle Metriken** die zu deinem Problem passen
- **Cross-Validation** für robuste Evaluierung
- **Teste auf Fairness** über verschiedene Gruppen
- **Dokumentiere** warum du bestimmte Modelle gewählt hast
- **Erkläre** deine Ergebnisse verständlich

### Reflexionsfragen:

1. **Welche Metrik ist für dein Projekt am wichtigsten? Warum?**
   
   ___________________________________

2. **Wie könntest du Bias in deinem Projekt-Modell erkennen?**
   
   ___________________________________

3. **Was würdest du bei Type I vs. Type II Errors priorisieren?**
   
   ___________________________________

---
## 📚 Weiterführende Ressourcen

### Empfohlene Materialien:

**Scikit-Learn Dokumentation:**
- [Model Evaluation](https://scikit-learn.org/stable/modules/model_evaluation.html)
- [Cross-Validation](https://scikit-learn.org/stable/modules/cross_validation.html)
- [Hyperparameter Tuning](https://scikit-learn.org/stable/modules/grid_search.html)

**Videos & Tutorials:**
- StatQuest: ROC and AUC Explained
- Google's ML Fairness Course
- EU AI Act Overview

**Tools:**
- [Fairlearn](https://fairlearn.org/) - Fairness Assessment & Mitigation
- [SHAP](https://shap.readthedocs.io/) - Model Explainability
- [What-If Tool](https://pair-code.github.io/what-if-tool/) - Interactive Model Analysis

**Ethical AI:**
- [EU Ethics Guidelines for Trustworthy AI](https://digital-strategy.ec.europa.eu/en/library/ethics-guidelines-trustworthy-ai)
- [Google's AI Principles](https://ai.google/principles/)
- [Microsoft's Responsible AI](https://www.microsoft.com/en-us/ai/responsible-ai)

---

### 🎉 Herzlichen Glückwunsch!

Du hast alle 4 Vorlesungstage abgeschlossen:
- ✅ Tag 2: Intro to Data Science
- ✅ Tag 4: Data Preparation
- ✅ Tag 6: Machine Learning
- ✅ Tag 8: Model Evaluation & Ethical AI

**Du bist jetzt bereit für das Abschlussprojekt! 🚀**

**Tipps für dein Projekt:**
1. Wähle ein Dataset das dich interessiert
2. Wende alle gelernten Techniken an (Preparation → ML → Evaluation)
3. Achte auf Fairness und ethische Aspekte
4. Dokumentiere deine Entscheidungen
5. Erzähle eine Geschichte mit deinen Daten

**Viel Erfolg! 💪**